In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
import random

Generate extra data for the model

In [28]:
def generate_body_type_data(body_type, num_samples):
    """
    Generates a specified number of data points for a given body type,
    based on anthropometric rules
    Args:
        body_type (str): The body type to generate data for (e.g., 'Bottom Hourglass')
        num_samples (int): The number of new samples to generate
    """
    generated_data = []
    
    # Define realistic ranges for measurements (in inches)
    bust_range = (30, 45)
    waist_range = (22, 35)
    hips_range = (32, 48)
    
    # Epsilon for safety in calculations
    safety_epsilon = 1e-6

    for _ in range(num_samples):
        bust = random.uniform(*bust_range)
        waist = random.uniform(*waist_range)
        hips = random.uniform(*hips_range)

        # Apply specific logic to fit the desired body type
        if body_type == 'Bottom Hourglass':
            # Hips slightly wider than bust, very defined waist, high_hip/waist ratio below 1.193
            bust = random.uniform(hips - 3.5, hips - 1.1)
            waist = random.uniform(bust - 9.0, bust - 5.0)
        elif body_type == 'Spoon':
            # Hips noticeably wider than bust, hips/waist diff < 9.0, bust/waist diff < 9.0
            # and a high_hip/waist ratio >= 1.193
            hips = random.uniform(bust + 3.6, bust + 9.9)
            bust = random.uniform(waist + 5.0, waist + 8.9)
        elif body_type == 'Hourglass':
            # Bust and hips are similar, waist is well-defined
            hips = random.uniform(bust - 1.0, bust + 1.0)
            waist = random.uniform(bust - 9.0, bust - 5.0)
        elif body_type == 'Rectangle':
            # All measurements are similar, waist is not well-defined
            bust = random.uniform(hips - 3.5, hips + 3.5)
            waist = random.uniform(bust - 5.0, bust - 0.1)
        elif body_type == 'Triangle':
            # Hips wider than bust, waist not well-defined
            hips = random.uniform(bust + 3.6, bust + 9.9)
            waist = random.uniform(hips - 9.0, hips - 5.0)
        elif body_type == 'Inverted Triangle':
            # Bust wider than hips, waist not well-defined
            bust = random.uniform(hips + 3.6, hips + 9.9)
            waist = random.uniform(bust - 9.0, bust - 5.0)
        elif body_type == 'Apple':
            # WHR >= 0.85
            waist = random.uniform(hips * 0.85, hips * 1.0)
            bust = random.uniform(hips * 0.8, hips * 1.2)
        
        # Ensure measurements are non-negative and somewhat realistic
        bust = max(bust, 25)
        waist = max(waist, 18)
        hips = max(hips, 28)
        
        generated_data.append({
            'bust': bust,
            'waist': waist,
            'hips': hips,
        })

    return pd.DataFrame(generated_data)

In [29]:
def generate_balanced_dataset(df, target_classes, num_per_class=500):
    """
    Generates synthetic data to balance the dataset
    Args:
        df (pd.DataFrame): The original DataFrame.
        target_classes (list): A list of the body types to generate data for.
        num_per_class (int): The number of samples to generate for each class.
    """
    original_classes = df['body_type'].unique()
    new_dataframes = []

    for body_type in target_classes:
        if body_type not in original_classes:
            print(f"Generating {num_per_class} samples for new class: {body_type}")
            generated_df = generate_body_type_data(body_type, num_per_class)
            generated_df['body_type'] = body_type
            new_dataframes.append(generated_df)
        else:
            current_count = df[df['body_type'] == body_type].shape[0]
            if current_count < num_per_class:
                samples_needed = num_per_class - current_count
                print(f"Generating {samples_needed} samples for class: {body_type}")
                generated_df = generate_body_type_data(body_type, samples_needed)
                generated_df['body_type'] = body_type
                new_dataframes.append(generated_df)

    # Combine the original data with the new synthetic data
    balanced_df = pd.concat([data] + new_dataframes, ignore_index=True)
    return balanced_df

In [31]:
if __name__ == '__main__':
    # Load your original DataFrame (from ML Body Type)
    data = pd.read_csv('body_type_merged_data.csv')
    
    # Classify the original data so we know the counts
    safety_epsilon = 1e-6
    data['whr'] = data['waist'] / (data['hips'] + safety_epsilon)
    data['bhr'] = data['bust'] / (data['hips'] + safety_epsilon)

    diff_bust_hips_small = 1.0
    diff_hips_bust_small = 3.6
    diff_waist_defined_bust = 9.0
    diff_waist_defined_hips = 10.0
    whr_apple_threshold = 0.85
    
    # Define conditions and choices in a clear, matching order
    conditions = [
        # Condition 1: Hourglass
        ( (data['bust'] - data['hips']).abs() <= diff_bust_hips_small) &
        ( (data['hips'] - data['bust']) < diff_hips_bust_small) &
        ( (data['bust'] - data['waist']) >= diff_waist_defined_bust) |
        ( (data['hips'] - data['waist']) >= diff_waist_defined_hips),
        
        # Condition 2: Bottom Hourglass
        ( (data['hips'] - data['bust']) >= diff_hips_bust_small) &
        ( (data['hips'] - data['bust']) < diff_waist_defined_hips) &
        ( (data['hips'] - data['waist']) >= diff_waist_defined_bust),
        
        # Condition 3: Top Hourglass
        ( (data['bust'] - data['hips']) > diff_bust_hips_small) &
        ( (data['bust'] - data['hips']) < diff_waist_defined_hips) &
        ( (data['bust'] - data['waist']) >= diff_waist_defined_bust),
        
        # Condition 4: Triangle
        ( (data['hips'] - data['bust']) >= diff_hips_bust_small) &
        ( (data['hips'] - data['waist']) < diff_waist_defined_bust),
        
        # Condition 5: Inverted Triangle
        ( (data['bust'] - data['hips']) >= diff_hips_bust_small) &
        ( (data['bust'] - data['waist']) < diff_waist_defined_bust),
        
        # Condition 6: Apple
        (data['whr'] >= whr_apple_threshold),

        # Condition 7: Rectangle
        ( (data['hips'] - data['bust']).abs() < diff_hips_bust_small) &
        ( (data['bust'] - data['waist']) < diff_waist_defined_bust) &
        ( (data['hips'] - data['waist']) < diff_waist_defined_hips)
    ]
    
    choices = [
        'Hourglass',
        'Bottom Hourglass',
        'Top Hourglass',
        'Triangle',
        'Inverted Triangle',
        'Apple',
        'Rectangle',
    ]
    data['body_type'] = np.select(conditions, choices, default='Undetermined')
    
    # Generate a balanced dataset
    all_body_types = [
        'Hourglass', 'Bottom Hourglass', 'Top Hourglass',
        'Triangle', 'Inverted Triangle', 'Apple', 'Rectangle'
    ]
    balanced_df = generate_balanced_dataset(data, all_body_types, num_per_class=500)
    
    # Calculate all the ratios on the final combined dataset
    # This ensures all rows, both original and generated, have these values
    balanced_df['whr'] = balanced_df['waist'] / (balanced_df['hips'] + safety_epsilon)
    balanced_df['bhr'] = balanced_df['bust'] / (balanced_df['hips'] + safety_epsilon)

    # Save the new, balanced dataset to a CSV file
    balanced_df.to_csv('balanced_df.csv', index=False)
    print("\nBalanced dataset generated and saved to 'balanced_data.csv'")
    print("\nNew dataset shape:", balanced_df.shape)
    print("\nNew class distribution:\n", balanced_df['body_type'].value_counts())


Generating 482 samples for class: Hourglass
Generating 499 samples for class: Bottom Hourglass
Generating 500 samples for new class: Top Hourglass
Generating 474 samples for class: Triangle
Generating 498 samples for class: Inverted Triangle
Generating 467 samples for class: Apple
Generating 485 samples for class: Rectangle

Balanced dataset generated and saved to 'balanced_data.csv'

New dataset shape: (3501, 8)

New class distribution:
 body_type
Apple                500
Triangle             500
Rectangle            500
Bottom Hourglass     500
Hourglass            500
Inverted Triangle    500
Top Hourglass        500
Undetermined           1
Name: count, dtype: int64


In [32]:
data = pd.read_csv('balanced_df.csv')
data.head()

,bust,hips,waist,height,gender,whr,bhr,body_type
0,8.0,8.0,7.0,24.0,female,0.875000,1.000000,Apple
1,8.0,19.0,14.0,63.0,female,0.736842,0.421053,Triangle
2,9.0,20.0,17.0,49.0,female,0.850000,0.450000,Triangle
3,9.0,45.0,41.0,60.0,female,0.911111,0.200000,Triangle
4,10.0,11.0,9.0,46.0,female,0.818182,0.909091,Rectangle


In [33]:
data.tail()

,bust,hips,waist,height,gender,whr,bhr,body_type
3496,38.613683,35.610157,33.732990,NaN,NaN,0.947286,1.084345,Rectangle
3497,32.278505,35.472493,30.268154,NaN,NaN,0.853285,0.909959,Rectangle
3498,37.257234,35.045253,33.100861,NaN,NaN,0.944518,1.063118,Rectangle
3499,39.217043,39.085475,36.526206,NaN,NaN,0.934521,1.003366,Rectangle
3500,46.569060,45.640218,44.870861,NaN,NaN,0.983143,1.020351,Rectangle


In [34]:
# drop the height and gender columns
data.drop(columns=['height','gender'], inplace=True)

In [35]:
data.shape

(3501, 6)

In [36]:
data.to_csv('data.csv', index=False)